In [4]:
import numpy as np
import time

## Testing Numpy Speed

In [2]:
def naive_add(x, y):
    assert len(x.shape) == 2
    assert x.shape == y.shape
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[i, j]
    return x

In [3]:
def naive_relu(x):
    assert len(x.shape) == 2
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] = max(x[i, j], 0)
    return x

In [7]:
x = np.random.random((200, 1000))
y = np.random.random((200, 1000))

t0 = time.time()
for _ in range(1000):
    z = x + y
    z = np.maximum(z, 0.)
print("Took: {0: .2f}s".format(time.time() - t0))

Took:  0.46s


In [8]:
t0 = time.time()
for _ in range(1000):
    z = naive_add(x, y)
    z = naive_relu(z)
print("Took: {0: .2f}s".format(time.time() - t0))

Took:  98.59s


## Testing Broadcasting

In [28]:
X = np.random.random((32, 10))
y = np.random.random((10))
X.shape
y.shape

(10,)

In [29]:
y = np.expand_dims(y, axis=0)
y.shape

(1, 10)

In [30]:
Y = np.concatenate([y] * 32, axis=0)

In [31]:
Y.shape

(32, 10)

In [32]:
x = np.random.random((64, 3, 32, 10))
y = np.random.random((32, 10))
z = np.maximum(x, y)

In [34]:
z.shape

(64, 3, 32, 10)

## Reshaping Tensors

In [38]:
x = np.array([[0., 1.],
            [2., 3.],
            [4., 5.]])
x.shape

(3, 2)

In [45]:
x.reshape((3, 2, 1))

array([[[0.],
        [1.]],

       [[2.],
        [3.]],

       [[4.],
        [5.]]])

## Gradient Tape in TensorFlow

In [47]:
import tensorflow as tf

In [58]:
x = tf.Variable(0.)
with tf.GradientTape() as tape:
    y = 2 * x + 3
grad_of_y_wrt_x = tape.gradient(y, x)
grad_of_y_wrt_x

<tf.Tensor: shape=(), dtype=float32, numpy=2.0>

In [54]:
x = tf.Variable(tf.random.uniform((2, 2)))
with tf.GradientTape() as tape:
    y = 2 * x + 3 
grad_of_y_wrt_x = tape.gradient(y, x)
grad_of_y_wrt_x

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 2.],
       [2., 2.]], dtype=float32)>

In [59]:
W = tf.Variable(tf.random.uniform((2, 2)))
b = tf.Variable(tf.zeros((2,)))
x = tf.random.uniform((2, 2)) 
with tf.GradientTape() as tape:
    y = tf.matmul(x, W) + b
grad_of_y_wrt_W_and_b = tape.gradient(y, [W, b])
grad_of_y_wrt_W_and_b

## Reimplementing MNIST in TF

In [98]:
class NaiveDense:
    def __init__(self, input_size, output_size, activation):
        self.activation = activation

        w_shape = (input_size, output_size)
        w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
        self.W = tf.Variable(w_initial_value)

        b_shape = (output_size,)
        b_initial_value = tf.zeros(b_shape)
        self.b = tf.Variable(b_initial_value)

    def __call__(self, inputs):
        return self.activation(tf.matmul(inputs, self.W) + self.b)

    @property
    def weights(self):
        return [self.W, self.b]


In [99]:
class NaiveSequential:
    def __init__(self, layers):
        self.layers = layers
    
    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
        return x
    
    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights += layer.weights
        return weights
    

In [100]:
model = NaiveSequential([
    NaiveDense(input_size=28 * 28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)
])
assert len(model.weights) == 4

In [101]:
import math

class BatchGenerator:
    def __init__(self, images, labels, batch_size=128):
        assert len(images) == len(labels)
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images) / batch_size)
    
    def next(self):
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.labels[self.index : self.index + self.batch_size]
        self.index += self.batch_size
        return images, labels


In [113]:
def one_training_step(model, images_batch, labels_batch):
    with tf.GradientTape() as tape:
        predictions = model(images_batch)
        
        per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(
            labels_batch, predictions
        )
        average_loss = tf.reduce_mean(per_sample_losses)
    gradients = tape.gradient(average_loss, model.weights)
    update_weights(gradients, model.weights)
    return average_loss

In [103]:
# learning_rate = 1e-3

# def update_weights(gradients, weights):
#     for g, w in zip(gradients, weights):
#         w.assign_sub(g * learning_rate)

from tensorflow.keras import optimizers
  
optimizer = optimizers.SGD(learning_rate=1e-3)
  
def update_weights(gradients, weights):
    optimizer.apply_gradients(zip(gradients, weights))

In [104]:
def fit(model, images, labels, epochs, batch_size=128):
    for epoch_counter in range(epochs):
        print(f"Epoch {epoch_counter}")
        batch_generator = BatchGenerator(images, labels, batch_size)
        for batch_counter in range(batch_generator.num_batches):
            images_batch, labels_batch = batch_generator.next()
            loss = one_training_step(model, images_batch, labels_batch)
            if batch_counter % 100 == 0:
                print(f"loss at batch {batch_counter}: {loss:.2f}")


In [114]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

train_labels.shape

fit(model, train_images, train_labels, epochs=10, batch_size=128)

Epoch 0
loss at batch 0: 0.62
loss at batch 100: 0.65
loss at batch 200: 0.57
loss at batch 300: 0.62
loss at batch 400: 0.68
Epoch 1
loss at batch 0: 0.59
loss at batch 100: 0.61
loss at batch 200: 0.54
loss at batch 300: 0.59
loss at batch 400: 0.66
Epoch 2
loss at batch 0: 0.57
loss at batch 100: 0.58
loss at batch 200: 0.51
loss at batch 300: 0.56
loss at batch 400: 0.64
Epoch 3
loss at batch 0: 0.54
loss at batch 100: 0.55
loss at batch 200: 0.49
loss at batch 300: 0.54
loss at batch 400: 0.62
Epoch 4
loss at batch 0: 0.52
loss at batch 100: 0.53
loss at batch 200: 0.47
loss at batch 300: 0.52
loss at batch 400: 0.60
Epoch 5
loss at batch 0: 0.51
loss at batch 100: 0.51
loss at batch 200: 0.45
loss at batch 300: 0.50
loss at batch 400: 0.59
Epoch 6
loss at batch 0: 0.49
loss at batch 100: 0.49
loss at batch 200: 0.43
loss at batch 300: 0.49
loss at batch 400: 0.58
Epoch 7
loss at batch 0: 0.48
loss at batch 100: 0.48
loss at batch 200: 0.42
loss at batch 300: 0.48
loss at batch 40

In [118]:
predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
print(f"accuracy: {matches.mean():.2f}")

accuracy: 0.86
